In [1]:
from enum import Enum
from pymongo import MongoClient#

In [41]:
 # Errors from Pymongo
from pymongo.errors import InvalidOperation

### Some Status Enums for MongoDB Manager Class

In [3]:
STATUS = Enum('STATUS', ['STARTING', 'IDLE', 'RUNNING', 'FAILED'])

### MongoDB Manager Class

In [ ]:
class MongoDBManager:
    """
    A MongoDB Manager. This avoids all the loose MongoDB connections and consoldiates them in one place.

    Note: Its resource intensive to instantiate a MongoDB connection
    """
    def __init__(self):
        """
        MongoDB Manager Initalizer
        ---------------------------
        uri: The MongoDB URI
        act_con: Active Connections
        init_connection: This creates the client object
        hist_connections_closed: Number of times a connection was closed.
        hist_connections_opened: Number of times a connection was opened.
        jobs_count: Number of jobs passed to MongoDB Manager
        """
        self.act_con = []
        self.upper_bound_con = 50 # We limit ourselves to 50 cocurrent conneciton
        self.hist_connections_closed = 0
        self.hist_connections_opened = 0
        self.jobs_count = 0
        self.job_queue = [] # For async purposes in the future...
        self.init_connection(uri) 
        
    def __str__(self):
        """
        Some useful metrics.
        """
        try:
            connection_list = []
            for connection in act_con:
               connection_list.append(self.ping(connection))
            return(
            f"========================\n"
            f"MongoDB Manager 1.0\n"
            f"Here are some Metrics!\n"
            f"========================\n"
            f"Connections: \n{connection_list} \n\n"
            f"Number of Active: {len(self.act_con)}\n"
            f"Number of Connections Closed: {self.hist_connections_closed}\n"
            f"Number of Open Connections: {self.hist_connections_opened}\n"
            f"Number of jobs processed: {self.jobs_count}\n"
            )
        except InvalidOperation as InvOp: # For any bad connections we see...
            connection_list.append({"BAD CONNECTION" : 0})
            continue # If we get bad or close connections
        return
        
    def ping(connection):
        """
        Pings the MongoDB server.
        """
        return connection.admin.command('ping')

    def init_connection(self, uri):
        """
        Attempts to initalize the connection. Should Instantiate a MongoDB Client object.
        """
        try:
            if (len(act_con) >= self.upper_bound_con):
                # We might run into this error if we parallize our containers that need each MongoDB manager
                raise Exception("Cannot create anymore connections! Aborting...")
            client = MongoClient(uri)
            act_con.append(client)
            self.hist_connections_opened += 1
            return
        except Exception as e:
            print(f"Original Error: {e}\n\n")
            raise Exception("[FATAL ERROR!] MongoDB Manager failed to initalize a MongoDB connection. Aborting...")
        return

    def force_close_connection(self, connection):
        """
        Attempts to force close/close the MongoDB connection.
        """
        try:
            if (not connection):
                print("[Warning] No client for this connection was found. No connection to close!")
                return
            connection.admin.command('ping') # We need to ping if the connection is alive
            connection.close();
            print(f"[INFO] Connection was successfully forcefully closed at {connection}")
            self.hist_connections_closed += 1
            
            return
        except InvalidOperation as InvOp:
            print(f"Original Error: {InvOp}\n\n")
            print("[Warning] Tried to close an already closed connection!")
        except Exception as e:
            print(f"Original Error: {e}\n\n")
            raise Exception("[FATAL ERROR!] MongoDB Manager failed close a MongoDB connection. Aborting...")
        return
                            
            
    def run_job(self, retry_times, connection, func, *args, **kwargs):
        """
        A wrapper for all MongoDB jobs. We can retry a "x" amount of times 
        -----
        retry_times: The number of times to retry for the connects
        connection: The PyMongo Connection you want to handle this job
        func: The function you want to execute
        """
        try:
            # Ping MongoDB
            # If MongoDB doesn't response, then we need to force a connection
            # We need to close our first connection and then open a new one...
            # If the connection is alive and well, then we just use as is, make sure we reset the timout to keep alive
            result = func(*args, **kwargs) # Run the with args
            # We need to force the connection to be closed as we are done.
            return result # Return anything if needed
        except Exception as e:
            return
        pass
        
        


### Some Stress Testing

In [42]:
db = MongoDBManager(
    uri="mongodb://localhost:27017"
)

# Spawn like 5 connections
db.init_connection()

In [43]:
print(db)

MongoDB Manager 0.1
Here are some Metrics!
Client Object: 
MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)
Number of Connections Closed: 0
Number of Open Connections: 1
Number of jobs processed: 0



In [39]:
db.force_close_connection()

[INFO] Connection was successfully forcefully closed at MongoClient(host=['ac-8xmnmpk-shard-00-00.5bib9wc.mongodb.net:27017', 'ac-8xmnmpk-shard-00-01.5bib9wc.mongodb.net:27017', 'ac-8xmnmpk-shard-00-02.5bib9wc.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, authsource='admin', replicaset='atlas-je9gce-shard-0', tls=True)


In [40]:
print(db)

MongoDB Manager 0.1
Here are some Metrics!
Client Object: 
MongoClient(host=['ac-8xmnmpk-shard-00-00.5bib9wc.mongodb.net:27017', 'ac-8xmnmpk-shard-00-01.5bib9wc.mongodb.net:27017', 'ac-8xmnmpk-shard-00-02.5bib9wc.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, authsource='admin', replicaset='atlas-je9gce-shard-0', tls=True)
Number of Connections Closed: 1
Number of Open Connections: 1
Number of jobs processed: 0



In [34]:
db.force_close_connection()

Original Error: Cannot use MongoClient after close


[Warning] Tried to close an already closed connection!
